In [1]:
data_var = '2024-01-19'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11204,2024-01-19,Brasil Nbb,19:00,Cerrado,Minas,3.44,1.28,156.5,1.81,1.89,5.5,2.08,1.63,vgfy5eWP,0.290698,0.781250,0.552486,0.529101,0.071948,0.4,0.6,NaN,NaN,204.542,52.980189,0.259019,120.820,29.206411,0.241735,174.886,128.536,123.75,119.88,0.0,0.0,0.0,0.0,0.647182,0.030578,0.171535,-0.43,-0.086,-28.372093,0.000000,0.0,0.000000,0.61,0.122,2.295082,0.000000,0.0,0.000000
11205,2024-01-19,Eua Nba,00:00,Minnesota Timberwolves,Memphis Grizzlies,1.13,6.54,213.5,1.87,1.99,-13.5,2.08,1.78,4C9WtQV4,0.884956,0.152905,0.534759,0.502513,0.037861,0.6,0.8,NaN,NaN,187.994,35.190554,0.187190,320.750,89.074216,0.277706,172.944,325.924,237.62,321.86,0.0,0.0,0.0,0.0,0.997509,0.043965,0.109913,1.31,0.262,0.496183,0.678570,0.8,0.121430,2.79,0.558,9.928315,0.364569,0.6,0.235431
11206,2024-01-19,Eua Nba,00:00,Sacramento Kings,Indiana Pacers,1.30,3.72,243.5,1.85,1.99,-8.5,2.02,1.83,r38ztpoB,0.769231,0.268817,0.540541,0.502513,0.038048,0.4,0.0,NaN,NaN,248.350,103.770348,0.417839,341.222,172.004442,0.504084,173.804,394.922,169.00,358.05,0.0,0.0,0.0,0.0,0.681752,0.051560,0.069792,-1.90,-0.380,-0.789474,0.687736,0.6,-0.087736,3.21,0.642,4.236760,0.378599,0.4,0.021401
11207,2024-01-19,Eua Nba,21:00,Charlotte Hornets,San Antonio Spurs,1.56,2.66,236.5,1.80,2.00,-5.5,2.00,1.71,jT7vu40H,0.641026,0.375940,0.555556,0.500000,0.016965,0.2,0.6,NaN,NaN,615.106,467.947356,0.760759,422.594,173.607590,0.410814,330.602,444.060,345.94,367.29,1.0,0.0,0.0,0.0,0.368634,0.074432,0.110545,-5.00,-1.000,-0.560000,0.361096,0.2,-0.161096,-3.32,-0.664,-2.500000,0.300605,0.2,-0.100605
11208,2024-01-19,Eua Nba,21:00,Orlando Magic,Philadelphia 76ers,2.74,1.54,221.5,1.83,1.97,4.5,2.03,1.91,UJ6rvOFN,0.364964,0.649351,0.546448,0.507614,0.014314,0.2,0.4,NaN,NaN,335.908,166.275415,0.495003,214.202,69.903089,0.326342,207.820,238.150,248.40,311.52,0.0,0.0,0.0,0.0,0.396508,0.052103,0.043072,-0.96,-0.192,-9.062500,0.640297,0.7,0.059703,-1.42,-0.284,-1.901408,0.552202,0.5,-0.052202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11319,2024-01-19,Finlândia I Divisão A,13:30,Honka,Kipina,1.10,6.04,168.5,1.82,1.88,-20.5,1.92,1.79,0jeP87LJ,0.909091,0.165563,0.549451,0.531915,0.074654,0.0,0.0,NaN,NaN,NaN,NaN,NaN,101.384,8.139919,0.080288,0.000,0.000,122.85,111.76,0.0,0.0,0.0,0.0,0.978461,0.022933,0.049555,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11320,2024-01-19,Finlândia I Divisão A,13:30,ToPo,Raholan Pyrkiva,1.19,4.32,166.5,1.85,1.85,-14.5,1.87,1.83,nm7spOrJ,0.840336,0.231481,0.540541,0.540541,0.071818,0.0,0.4,NaN,NaN,128.616,40.081251,0.311635,141.890,32.978256,0.232421,0.000,298.044,110.00,123.28,0.0,0.0,0.0,0.0,0.803355,0.000000,0.015289,0.00,0.000,inf,0.000000,0.0,0.000000,-0.63,-0.126,-26.349206,0.000000,0.0,0.000000
11321,2024-01-19,Suécia Basketligan,15:04,Sodertalje,Borås,3.02,1.35,164.5,1.84,1.86,2.5,2.05,1.65,APGdvo6l,0.331126,0.740741,0.543478,0.537634,0.071867,0.4,0.8,NaN,NaN,197.756,111.786237,0.565274,148.718,73.539014,0.494486,154.060,169.344,168.30,118.80,0.0,0.0,0.0,0.0,0.540443,0.007644,0.152888,0.22,0.044,45.909091,0.000000,0.0,0.000000,-0.71,-0.142,-2.464789,0.000000,0.0,0.000000
11322,2024-01-19,Taiwan Sbl,08:30,Yulon Luxgen Dinos,Bank of Taiwan,1.59,2.26,153.5,1.86,1.84,-4.5,1.94,1.77,xCp1gszn,0.628931,0.442

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Minnesota Timberwolves,Memphis Grizzlies,213.5,1.87,0.9647,Over
1,00:00,Eua Nba,Sacramento Kings,Indiana Pacers,243.5,1.85,0.8916,Over
2,21:00,Eua Nba,Orlando Magic,Philadelphia 76ers,221.5,1.83,0.9938,Over
3,16:00,Europa Euroliga,Monaco,Real Madrid,163.5,1.85,1.0000,Over
